### 1. the distribution of the original weight of SDXL-Turbo

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline, UNet2DModel, UNet2DConditionModel, LCMScheduler
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def get_model(model_id: str="Lykon/dreamshaper-7", cache_dir: str="/share/public/diffusion_quant/huggingface/hub", type: str="lcm_lora"):
    print(f"the weight is from {model_id}")
    # pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
    if 'xl' in type:
        pipe = StableDiffusionXLPipeline.from_pretrained(model_id, cache_dir=cache_dir)
    else:
        pipe = StableDiffusionPipeline.from_pretrained(model_id, cache_dir=cache_dir)

    # print(type(model))
    if 'lcm' in type:
        pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
    if 'lora' in type:
        # load and fuse lcm lora
        adapter_id = "latent-consistency/lcm-lora-sdv1-5"
        pipe.load_lora_weights(adapter_id)
        pipe.fuse_lora()

    model = pipe.unet
    # model.cuda()
    # model.eval()
    return model

In [ ]:
def get_weights(model, layer_name):
    for name, module in model.named_parameters():
        # print(module.data.shape)
        if name == layer_name:
            weights = module.data
            # 将权重张量转换为numpy数组，并取绝对值
            weights_np = np.abs(weights.detach().numpy())
            
            # 获取输入通道和输出通道的数量
            out_channels, in_channels = weights_np.shape[0:2]
            
            if len(weights_np.shape)==4:
                # 在H、W维度上取均值
                weights_np = weights_np.mean(axis=(2, 3))
                # reshape成（out_channel，in_channel）维度的张量
                weights_np = weights_np.reshape(out_channels, in_channels)
            return weights_np

In [ ]:
model_id = "stabilityai/sdxl-turbo"
model = get_model(model_id=model_id, type="sdxl_turbo")

In [ ]:
weight_id = "down_blocks.1.attentions.0.proj_out.weight"  # 观察哪一层的权重
weights_np = get_weights(model, weight_id)

#### 3D view

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as colors
import os

# weights_np[49:60,0:100] = 0.7

# 创建一个新的图形和3D子图
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# ax = plt.axes(projection='3d')

# 绘制3D表面
weights_np = weights_np
cs = weights_np
surf = ax.plot_surface(_X, _Y, weights_np, cmap='coolwarm', vmin=np.min(weights_np), vmax=np.max(weights_np)/2.5)
# points = ax.scatter3D(_X, _Y, weights_np, c=cs, s=0.5, cmap='coolwarm', vmin=np.min(weights_np), vmax=np.max(weights_np))

# # 创建一个自定义的归一化对象
# norm = colors.Normalize(vmin=np.min(weights_np), vmax=np.max(weights_np), clip=True)
# # 绘制3D表面
# surf = ax.plot_surface(_X, _Y, weights_np, cmap='coolwarm', norm=norm)


# 设置轴标签
ax.set_xlabel('Output Channels')
ax.set_ylabel('Input Channels')
ax.set_zlabel('Absolute Weights')

# ax.set_zlim([np.min(weights_np), np.max(weights_np)])
ax.set_zlim([-np.max(np.abs(weights_np)), np.max(np.abs(weights_np))])
print(np.max(np.abs(weights_np)))

# 添加颜色条
# fig.colorbar(points)
fig.colorbar(surf)
# ax.view_init(elev=20, azim=20)
# # 设置面板颜色
# ax.w_xaxis.set_pane_color((1, 0.1, 1, 1))  # 设置x轴面板为白色
# ax.w_yaxis.set_pane_color((1, 0.1, 1, 1))  # 设置y轴面板为白色
# ax.w_zaxis.set_pane_color((1, 0.1, 1, 1))  # 设置z轴面板为白色

# 保存图像到文件
os.makedirs('./act_distribution_img/',exist_ok=True)
plt.savefig('./act_distribution_img/weights_distribution_3d.png')

# 显示图形
# plt.show()


In [ ]:
weight_id = "down_blocks.0.resnets.1.conv1.weight"  # 观察哪一层的权重
weights_np = get_weights(model, weight_id)
out_channels, in_channels = weights_np.shape

# --------- Plotly Equivalent of this Plot -------

import plotly.graph_objects as go
import numpy as np
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# Assuming you have the meshgrid arrays _X, _Y, and weights_np defined

fig = go.Figure()

colormap_span = 3
channel_subsample = None

if channel_subsample is not None:
    out_channels = channel_subsample
    in_channels = channel_subsample
    weights_np_ = weights_np[:channel_subsample, :channel_subsample]
else:
    weights_np_ = weights_np
    
# 创建X，Y meshgrid
_x = np.arange(out_channels)
_y = np.arange(in_channels)
_X, _Y = np.meshgrid(_x, _y)

_X = _X.T
_Y = _Y.T

surf = go.Surface(x=_X, y=_Y, z=weights_np_, colorscale='IceFire', cmin=np.min(weights_np_), \
                  cmax=np.max(weights_np_),opacity=0.5)
fig.add_trace(surf)

# Set z-limits
fig.update_layout(scene=dict(zaxis=dict(range=[-np.max(np.abs(weights_np)), np.max(np.abs(weights_np))])),
                  width=800, height=800,  # Set the figure size
                 )

# Print max absolute value of weights_np
print(np.max(np.abs(weights_np_)))

# Add color bar
fig.update_layout()

# Show the plot
iplot(fig)


#### 2D view

In [ ]:
weight_id = "down_blocks.0.resnets.1.conv1.weight"  # 观察哪一层的权重
for name, module in model.named_parameters():
    # print(module.data.shape)
    if name == weight_id:
        weights = module.data

import torch
import matplotlib.pyplot as plt

# 是否选择逐channel去看分布
channel_type = None
# 看第几个channel
view_channel = 1

if channel_type=='output_channel':
    tensor = weights[view_channel].reshape(-1)
elif channel_type=='input_channel':
    tensor = weights[:,view_channel].reshape(-1)  # view不能处理内存不连续的张量
else:
    # 假设您的张量是 tensor
    tensor = weights.reshape(-1)

# 将张量转换为numpy数组
numpy_array = tensor.numpy()

# 使用matplotlib的hist函数绘制分布图
plt.hist(numpy_array, bins='auto', label=weight_id+': '+channel_type+'.'+str(view_channel) if  channel_type is not None else weight_id)  
# 'auto'会自动选择最佳的bins数量

plt.title('distribution')
plt.xlabel('value')
plt.ylabel('freq')

plt.legend()
plt.yscale('log')

plt.show()


#### Box-Plot

In [ ]:
import torch
import matplotlib.pyplot as plt

# 假设您的张量是 tensor
tensor = weights  # 这是一个4阶张量，第0阶和第1阶分别代表输出通道和输入通道

# 假设 channel_type 是 'output' 或 'input'
channel_type = 'output_channel'  # 请将此行替换为实际的 channel_type

if channel_type == 'output_channel':
    channels = [tensor[i].numpy().flatten() for i in range(tensor.shape[0])]
elif channel_type == 'input_channel':
    channels = [tensor[:, j].numpy().flatten() for j in range(tensor.shape[1])]

plt.figure(figsize=(22, 10))

# 使用matplotlib的boxplot函数绘制箱状图
bplot = plt.boxplot(channels, patch_artist=True, notch=True, vert=1)

colors = ['pink', 'lightblue', 'lightgreen']
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)

# 添加图例
plt.legend([bplot["boxes"][0]], [weight_id+': '+channel_type], loc='upper right')

plt.title('box-plot')
plt.xlabel('channel_index')
plt.ylabel('range')

plt.show()


### 2. the comparison between the sdxl-turbo and the original sdxl

In [ ]:
model_id_non_turbo = "stabilityai/stable-diffusion-xl-base-1.0"
model = get_model(model_id=model_id_non_turbo, type="sdxl")
weight_id_non_turbo = weight_id  # 观察哪一层的权重
weights_non_turbo = get_weights(model, weight_id)

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

# 是否选择逐channel去看分布
channel_type = None
# 看第几个channel
view_channel = 1

if channel_type=='output_channel':
    flat_tensor1 = weights[view_channel].reshape(-1)
    flat_tensor2 = weights_non_turbo[view_channel].reshape(-1)
elif channel_type=='input_channel':
    flat_tensor1 = weights[:,view_channel].reshape(-1)  # view不能处理内存不连续的张量
    flat_tensor2 = weights_non_turbo[:,view_channel].reshape(-1)  # view不能处理内存不连续的张量
else:
    # 假设您的张量是 tensor
    flat_tensor1 = weights.reshape(-1)
    flat_tensor2 = weights_non_turbo.reshape(-1)

# 将张量转换为numpy数组
numpy_array1 = flat_tensor1.numpy()
numpy_array2 = flat_tensor2.numpy()

# 使用matplotlib的hist函数绘制分布图，设置weights参数
plt.hist(numpy_array2, bins=3000, alpha=0.5, label='sdxl_'+weight_id_non_turbo+': '+channel_type+'.'+str(view_channel) if  channel_type is not None else 'sdxl_'+weight_id_non_turbo)
plt.hist(numpy_array1, bins=3000, alpha=0.5, label='sdxl_turbo_'+weight_id+': '+channel_type+'.'+str(view_channel) if  channel_type is not None else 'sdxl_turbo_'+weight_id)


# 设置y轴的刻度为对数刻度
plt.yscale('log')
plt.title('distribution')
plt.xlabel('value')
plt.ylabel('freq')

plt.legend()
plt.show()




In [ ]:
import torch
import numpy as np

def compute_weight_info(weights):
# 假设您的张量是 tensor
    tensor = weights.reshape(-1)

    # 将张量转换为numpy数组
    numpy_array = tensor.numpy()

    # 计算熵
    counts, _ = np.histogram(numpy_array, bins=3000)
    p = counts / counts.sum()
    p = p+1e-10
    entropy = -np.sum(p * np.log(p))

    # 计算离群值的分布程度
    # 基于四分位数范围（IQR）的离群值检测方法
    q1, q2 = np.percentile(numpy_array, [95, 5])
    iqr = q1 - q2
    threshold = 1.5 * iqr
    outliers = numpy_array[(numpy_array < (q1 - threshold)) | (numpy_array > (q2 + threshold))]
    outlier_count = len(outliers) / len(numpy_array)

    # 计算方差
    variance = np.var(numpy_array)

    print(f'entropy: {entropy}')
    print(f'outlier_percent: {outlier_count*1e3}x1^(-3)')
    print(f'variance: {variance*1e5}x10^(-5)')
    return entropy, outlier_count, variance

In [ ]:
compute_weight_info(weights)

In [ ]:
def get_weights_info(model):
    weight_names = []
    entropys = []
    variances = []
    outlier_freqs = []
    for name, module in model.named_parameters():
        print(module.data.shape)
        # print(line, type(line))
        weight_names.append(name)
        entropy, variance, outlier_freq = compute_weight_info(module.data)
        entropys.append(entropy)
        variances.append(variance)
        outlier_freqs.append(outlier_freq)
    return weight_names, entropys, variances, outlier_freqs

In [ ]:
weight_names, entropys, variances, outlier_freqs = get_weights_info(model)
# 使用matplotlib来绘制折线图
plt.figure(figsize=(80, 6))
plt.plot(weight_names, entropys, marker='o')
plt.xlabel('weight of layers')
plt.ylabel('entropys')
plt.title('entropys for weights of different layers')
plt.grid(True)
plt.xticks(rotation=90, fontsize=2)
plt.show()

In [ ]:
plt.figure(figsize=(80, 8))
plt.plot(weight_names, entropys, marker='o')
plt.xlabel('Blocks')
plt.ylabel('SQNR (dB)')
plt.title('SQNR for different blocks')
plt.grid(True)
plt.xticks(rotation=90, fontsize=6)
plt.show()

In [ ]:
plt.figure(figsize=(80, 8))
plt.plot(weight_names, entropys, marker='o')
plt.xlabel('Blocks')
plt.ylabel('SQNR (dB)')
plt.title('SQNR for different blocks')
plt.grid(True)
plt.xticks(rotation=60, fontsize=6)
plt.show()